# Scraping Code for AFE
The code below scrapes both the places and finds from the AFE website. afe_places require Selenium to work, is simpler, but is finicky in terms of start up. The finds reads the bare HTML, but due to variation in HTML templates for different kinds of finds, it involves a multipart conditional and is consequently rather long.

Strictly speaking, scraping would not be at all needed, since the DAI-AFE website offers a CSV export of its data that exports everything in one go, provided you search without any parameters. The problem is that, while this export provides the find numbers, it does not provide place or hoard numbers, and so prevents the user from re-assembling the data. The virtue of the scraping procedure below is that it also gets those particular numbers.

And THIS might not have been a problem either, since one could join on distinct strings and get to the numbers the other way around. But there are doubles in the afe_places df, such that a coin from that place would not know which entry to choose based on the provided value. These all have two entries in the places df: 

Gudensberg  
Karlstadt   
Vockerode   
Berghofen   
Dissen      
Hamm        
Haffen      
Poppenhausen
Stockum     
Limburg       

In [1]:
# These are here in case you are debugging the scrapers. They are re-imported in the data cleaning section.
import pandas as pd
import numpy as np

## The scrapers
The import libraries below are common more or less for all the scrapers. Two of three use Selenium. Detailed finds doesn't require it.

In [ ]:
import matplotlib.pyplot as plt
from pylab import savefig

%matplotlib inline

import datetime as dt
import io
import ast

import urllib3
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen, Request
import re
import datetime
import time

def setting_up(sales_url):
  headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
  req = Request(url=sales_url, headers=headers) 
  html = urlopen(req).read().decode('utf-8')
  soup = BeautifulSoup(urlopen(req).read())
  return soup

#!pip install selenium
#!apt-get update # to update ubuntu to correctly run apt install
#!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys

#!pip install webdriver_manager
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())
#sys.path.insert(0,'/opt/homebrew/Caskroom/chromedriver')
#from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


from selenium.webdriver.common.by import By

### Place Scraper

In [ ]:
# This code works to grab what is needed from the places website using Selenium
# In addition to Lee's code, I have added a tester that continues the loop
# in cases of failure. 
# After every loop, I am also outputing a new copy of the csv, which will help with
# debug in cases where this fails, say, 1500 rows in.

for i in range(1, 10):
    link = 'http://afe.dainst.org/place?afeid=' + str(i)

    try:
        driver.maximize_window()
        driver.implicitly_wait(1000)
        driver.get(link)
        driver.implicitly_wait(1000)
        driver.find_element(By.CLASS_NAME,'lclinks')

        html = driver.page_source
        #print(pd.read_html(html)[0])
        places_temp = pd.read_html(html)[0].set_index('Name')
        #print(places_temp.loc['ID'][0])

        new_line = [places_temp.loc['ID'][0], places_temp.loc['Name'][0],places_temp.loc['Längen- und Breitengrad'][0].split(',')[0],places_temp.loc['Längen- und Breitengrad'][0].split(',')[1]]
        places.loc[i] = new_line
        places.to_csv('afe_places.csv', index=False)
    except:
        continue

### "Detailed Result" Scraper
This scrapes what is essentially the report of individual coins. This is where the majority of our information comes from. It is a difficult scrape. We aren't using Selenium, but instead just targeting text patterns and grabbing whatever comes up in them. This results in some incredibly messy columns, occasionally grabbing the entire HTML text and sticking it into individual columns. I have, through Regex, tried to tame these. The occasional outlier gets through, even by the end, which you will see me weed out of the results in mint conversion stage of data cleaning (not even in this section... way down the line).

In [ ]:
# This works, and covers a number of different variations in the HTML template used to generate pages
# So far used on #1-5000

start_id = 15001
end_id = 18000

cols = ['ID', 'place_id', 'afe_find_id', 'Location', 'Status', 'Denomination', 'Issuer', 'Mint', 'Date', 'References', 'Remarks', 'Bibliography']
AFE = pd.DataFrame(columns=cols)

while start_id <= end_id:
    url = 'http://afe.dainst.org/detailedresult?l=en&link=' + str(start_id)
    links = setting_up(url).find_all('a')
    place_link = str(links[0])
    blank = '<a href="http://afe.dainst.org/coin?afeid=&amp;l=en" target="_blank">http://afe.dainst.org/coin?afeid=</a>'
    
    # A big problem in the AFE implementation is that links come in various orders.
    # We had solved this by accounting for fixed places in which links came
    # But these turned out not to be so fixed. So I've put in a simple 
    # loop that determines the right link for each entry. This should fix 
    # "invalid literal" errors that pop up when you pass the wrong formatted link
    # down the pipeline

    
    image = '.jpg'
    if place_link == blank:
        start_id += 1
        print(start_id)
        time.sleep(0.2)
        continue
    elif image in place_link:
        # The link number differs for image pages than not (say AFE 4 or 5). This is one difference.
        #place_link = str(links[2])
        #print(place_link)
        
        marker = "place?afeid="
        for i in links:
            if marker in str(i):
                place_link = str(i)
            else:
                continue

        temp = setting_up(url).get_text()
        #print(temp)

        loc = temp.find('Location')
        sta = temp.find('Status')
        deno = temp.find('Denomination')
        iss = temp.find('Issuer')
        mint = temp.find('Mint')
        dat = temp.find('Date')
        ref = temp.find('References')
        rem = temp.find('Remarks')
        bib = temp.find('Bibliography')
        end_text = temp.find('Export CSV')-22
        #print(loc,sta,deno,iss,mint,dat,ref,rem,bib,end_text)

        # One major problem is that various fields wind up being empty in the AFE. When they are
        # It breaks the code, since the fields simply disappear. These produce a "-1" value when
        # Evaluatd by the above "find" functions. The code below is a diagnostic to show which fields 
        # Are empty
        #fields = [loc,sta,deno,iss,mint,dat,ref,rem,bib,end_text]

        #for n, i in enumerate(fields):
        #    if i == -1:
        #        fields[n] = 'empty'
        #    else:
        #        continue


        # The problem with image pages is that some have status fields and others do not. It is inconsistent.
        # so we need to build in a conditional for this. this is done for each variable below, where it is 
        # known to be a problem
        if sta != -1:
            loc_string = temp[loc+8:sta-1]
            sta_string = temp[sta+6:deno]
        else:
            loc_string = temp[loc+8:deno]
            sta_string = 'No status'

        deno_string = temp[deno+12:iss]

        if temp.find('Issued for') > 0: iss_string = temp[iss+6:temp.find('Issued for')]
        else: iss_string = temp[iss+6:mint]

        min_string = temp[mint+4:dat]

        if ref > 0: dat_string = temp[dat+4:ref]
        else: dat_string = temp[dat+4:rem]

        if (temp.find('Obv./Rev.') > 0) and (ref > 0): ref_string = temp[ref+10:temp.find('Obv./Rev.')]
        elif (temp.find('Obv./Rev.') == -1): ref_string = ''
        elif (ref > 0): ref_string = temp[ref+10:rem]
        else: ref_string = 'No reference'

        if bib > 0: 
            rem_string = temp[rem+7:bib]
        elif bib == -1: 
            rem_string = 'No remarks'

        bib_string = temp[bib+12:end_text]
        #print(loc_string, sta_string, deno_string, iss_string, min_string, dat_string, ref_string, rem_string, bib_string, sep='; ')

        #print(setting_up(url).find('#place?afeid='))
        #links = setting_up(url).find_all('a')

        place_start = place_link.find('place?afeid=') + 12
        place_end = place_link.find('" tar')
        #print(place_start, place_end)

        place_id = int(place_link[place_start:place_end])

        #The same problem above is replicated in the find_spot link position.
        #Also created an "invalid literal" error
        find_marker = "findspot?afeid="
        for i in links:
            if find_marker in str(i):
                find_link = str(i)
            else:
                continue
        #find_link = str(links[2])
        link_start = find_link.find('findspot?afeid=') + 15
        link_end = find_link.find('" tar')
        link_id = int(find_link[link_start:link_end])


        #print(place_id, link_id, sep='; ')
        new_line = [start_id, place_id, link_id, loc_string, sta_string, deno_string, iss_string, min_string, dat_string, ref_string, rem_string, bib_string]
        AFE.loc[start_id] = new_line

        start_id += 1
        print(start_id)
        time.sleep(0.2)
        continue
    else:
        #url = 'http://afe.dainst.org/detailedresult?l=en&link=' + str(start_id)
        temp = setting_up(url).get_text()
        loc = temp.find('Location')
        sta = temp.find('Status')
        deno = temp.find('Denomination')
        iss = temp.find('Issuer')
        mint = temp.find('Mint')
        dat = temp.find('Date')
        ref = temp.find('References')
        rem = temp.find('Remarks')
        bib = temp.find('Bibliography')
        end_text = temp.find('Export CSV')-22

        loc_string = temp[loc+8:sta-1]
        sta_string = temp[sta+6:deno]
        deno_string = temp[deno+12:iss]
        if temp.find('Issued for') > 0: iss_string = temp[iss+6:temp.find('Issued for')]
        else: iss_string = temp[iss+6:mint]
        min_string = temp[mint+4:dat]
        if ref > 0: dat_string = temp[dat+4:ref]
        else: dat_string = temp[dat+4:rem]
        if (temp.find('Obv./Rev.') > 0) and (ref > 0): ref_string = temp[ref+10:temp.find('Obv./Rev.')]
        elif (ref > 0): ref_string = temp[ref+10:rem]
        else: ref_string = ''
        rem_string = temp[rem+7:bib]
        bib_string = temp[bib+12:end_text]
        #print(loc_string, sta_string, deno_string, iss_string, min_string, dat_string, ref_string, rem_string, bib_string, sep='; ')


        #print(setting_up(url).find('#place?afeid='))
        links = setting_up(url).find_all('a')
        #place_link = str(links[0])
        place_start = place_link.find('place?afeid=') + 12
        place_end = place_link.find('" tar')
        place_id = int(place_link[place_start:place_end])

        find_link = str(links[1])
        link_start = find_link.find('findspot?afeid=') + 15
        link_end = find_link.find('" tar')
        link_id = int(find_link[link_start:link_end])


        #print(place_id, link_id, sep='; ')
        new_line = [start_id, place_id, link_id, loc_string, sta_string, deno_string, iss_string, min_string, dat_string, ref_string, rem_string, bib_string]
        AFE.loc[start_id] = new_line

        start_id += 1
        time.sleep(0.2)
        print(start_id)
AFE.to_csv('afe_detailed_results.csv', index=False)

### Find Scraper (Fund Ort)
This one was pretty simple. Another instance of Selenium scraping.

In [ ]:
# All of the previous scraping is not sufficient because we are missing find categories (hoards, etc.)
# We need to scrape the fundort pages too
# This requires Selenium :-(

# This code works to grab what is needed from the places website using Selenium
# In addition to Lee's code, I have added a tester that continues the loop
# in cases of failure. 
# After every loop, I am also outputing a new copy of the csv, which will help with
# debug in cases where this fails, say, 1500 rows in.
#!pip install webdriver_manager

# We ran into some problems, and it had to do with the reset timing on the Selenium browser. 
# I built a test function that basically told the program to continue a loop if it did not 
# resolve within 5 seconds (thus getting past non-conforming URLs). 
# But the browser had to be reset for the next step too---except the browser auto-reset 
# was 1000 seconds and nothing about our reset test triggered this. 
# Thus all future loops resolved without any input from the browsers (they were all failed loops). 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

from selenium.webdriver.common.by import By

# There are a lot of empty spaces in the finds database.
# They take a long time to resolve, so any find that takes longer than
# A certain amount of time is an error. We build in a timer to resolve
# These errors
import signal
from contextlib import contextmanager

class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

# You have to create the DF in advance, because if the try function fails, subsequent
# attempts, including in except will have no df to latch onto.
# Lee did not need this in the places scrape, since there is a successful place at 
# loc #1. There is not one here.
data = [['','','','']]
finds = pd.DataFrame(data, columns=['ID', 'Name', 'Fundkategorie', 'Link'])        

for i in range(1,4018):
    link = 'http://afe.dainst.org/findspot?afeid=' + str(i)
    driver.maximize_window()
    driver.implicitly_wait(4)
    driver.get(link)
    driver.implicitly_wait(4)

    
    #print(pd.read_html(html)[0])
    try:
        with time_limit(5):
            driver.find_element(By.CLASS_NAME,'lclinks')

            html = driver.page_source

            finds_temp = pd.read_html(html)[0].set_index('Name')
            #print(places_temp.loc['ID'][0])

            new_line = [finds_temp.loc['ID'][0], finds_temp.loc['Name'][0],finds_temp.loc['Fundkategorie'][0],finds_temp.loc['Link'][0]]
            finds.loc[i] = new_line
            print(i, "Success!")
            finds.to_csv('afe_fundort.csv', index=False)
    except:
        new_line = ['', '','','']
        finds.loc[i] = new_line
        print(i, "Empty")
        continue

# Converting Scraped Data to Our Format
We start with the Detailed Finds report and work off of that, as that is the main source of information.

In [1]:
import numpy as np
import pandas as pd
import datetime
import re

pd.set_option('display.max_columns', 5000) 
pd.set_option('display.max_rows', 5000)
dai = pd.read_csv('afe_detailed_results.csv')

# Some elements messing up the map
dai['Location'] = dai['Location'].str.replace('(','')
dai['Location'] = dai['Location'].str.replace(')','')
dai['Location'] = dai['Location'].str.replace('?','')
dai['Location'] = dai['Location'].str.replace('"','')

# Replace NaNs in the YearFound column with 1700, since blank years prevents those
# rows from being summed along with the other ones

#dai['YearFound'] = dai['YearFound'].fillna(1700)
dai['Status'] = dai['Status'].fillna('Einzelfund_FC')

# There are many values in dai['Date'] that simply won't yield a date range. So we
# Are doing some Regex work here.

# This is a an error that occurs when nothing is captured for the Date value
# It throws in the entire HTML output of the scraper -- not useful
# We replace that with "malformed"
for row, i in enumerate(dai['Date']):
    dai['Date'].loc[row] = re.sub("[.|\n|\W|\w]*Detailed Result[.|\n|\W|\w]*", "Malformed", i)

# This replaces all '\n' entries in bad values that do not conform to the above
# The '\n' interferes with identifying date ranges, setting up a succesful
# Targeting operation in the next loop
for row, i in enumerate(dai['Date']):
    dai['Date'].loc[row] = re.sub("(.*)(\\n)(.*)", "\\1\\3", i)

# Need to replace 'bis' with '-'
for row, i in enumerate(dai['Date']):
    dai['Date'].loc[row] = re.sub("bis", "-", i)

# What follows is a pretty straightforward series of transformations of bad formats
for row, i in enumerate(dai['Date']):
    dai['Date'].loc[row] = re.sub("(LT)(.*)", "Unclear Periodization", i)
for row, i in enumerate(dai['Date']):
    dai['Date'].loc[row] = re.sub("(nach\s)(\d+)", "\\2-750", i)
for row, i in enumerate(dai['Date']):
    dai['Date'].loc[row] = re.sub("(\()(\d+-\d+)", "\\2", i)
for row, i in enumerate(dai['Date']):
    dai['Date'].loc[row] = re.sub("Zeitstellung unbekannt*", "Date Unknown", i)

/var/folders/j3/n5w99lgd16773rvsbzs4mmdh0000gn/T/ipykernel_5986/694732682.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  dai['Location'] = dai['Location'].str.replace('(','')
/var/folders/j3/n5w99lgd16773rvsbzs4mmdh0000gn/T/ipykernel_5986/694732682.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  dai['Location'] = dai['Location'].str.replace(')','')
/var/folders/j3/n5w99lgd16773rvsbzs4mmdh0000gn/T/ipykernel_5986/694732682.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  dai['Location'] = dai['Location'].str.re

In [2]:
# This loop deals exclusively with transforming century ranges into hard date ranges
# It relies exlusively on re

for row, i in enumerate(dai['Date']):
    # there are a few common kinds of tags for dross. Everything after these keywords is garbage
    entry = re.sub("(.*)(Bibliography)(.*)", "\\1", i)
    entry = re.sub("(.*)(Secondary)(.*)", "\\1", entry)
    entry = re.sub("(.*)(Weight)(.*)", "\\1", entry)
    entry = re.sub("(.*)(Obv)(.*)", "\\1", entry)
    entry = re.sub("(.*)(Peculiarities)(.*)", "\\1", entry)
    # After this, we have some conditionals. These account for differences in form btw
    # Single century dates, as well as BCE dates (those with only BCE are cut entirely, and
    # those that straddle BCE/CE are assigned a 1-??? date range)

    if re.search("Jh.|Jahrhundert|\d+\.", i) != None:
        try:
            # The len(???) conditional assesses how many numbers are present in a field.
            # If 2, then it is a range, if 1 then not
            if len(re. findall("\d+", entry)) == 1 and re.search("v\.\sChr\.", i) != None:
                century_start = '0'
                century_end = '0'
            elif len(re. findall("\d+", entry)) == 1:
                century = re.sub("(.*)(\d+)(.*)", "\\2", entry)
                century_start = int(century) - 1
                century_start = str(century_start) + "00"
                if century_start == "000":
                    century_start = "1"
                century_end = int(century) - 1
                century_end = str(century_end) + "99"
            elif len(re. findall("\d+", entry)) == 2 and re.search("Jahrhundert|Jh", i) == None:
                century_start = re.sub("(\d+)(.*?)(\d+)(.*)", "\\1", entry)
                century_end = re.sub("(\d+)(.*?)(\d+)(.*)", "\\3", entry)
                century_start = int(century_start) - 1
                century_start = str(century_start) + "00"
                if century_start == "000":
                    century_start = "1"
                century_end = int(century_end) - 1
                century_end = str(century_end) + "99"
            elif len(re. findall("\d+", entry)) == 2 and re.search("v\.\sChr\.", i) == None:
                century_start = re.sub("(\d+)(.*?)(\d+)(.*)", "\\1", entry)
                century_end = re.sub("(\d+)(.*?)(\d+)(.*)", "\\3", entry)
                century_start = int(century_start) - 1
                century_start = str(century_start) + "00"
                if century_start == "000":
                    century_start = "1"
                century_end = int(century_end) - 1
                century_end = str(century_end) + "99"
            elif len(re. findall("\d+", entry)) == 2 and re.search("v\.\sChr\.", i) != None and re.search("n\.\sChr\.", i) != None:
                century_start = '1'
                century_end = re.sub("(\d+)(.*?)(\d+)(.*)", "\\3", i)
                century_end = int(century_end) - 1
                century_end = str(century_end) + "99"
            else:
                century_start = "0"
                century_end = "0"

            if century_end == '099':
                century_end = '99'

            date = century_start + "-" + century_end
            dai['Date'].loc[row] = date
        except:
            continue
    else:
        continue

In [3]:
# This is looking for any lone century dates (e.g., 1. Jh.) by looking for 'Jh.' within a certain number
# of characters of 1 (or whatever digit it may be). It seems to work pretty well.
for row, i in enumerate(dai['Date']):
    if re.search("(\d+)(\.)(?![.!?] )\W+(?:\w+(?![.!?] )\W+){1,2}?(Jh.){1,2}?(?![.!?] )\W+(?:\w+(?![.!?] )\W+)", i) != None:
        print(row, i)
        century = re.sub("(\d+)(\.)(.*)", "\\1", i)
        century = int(century) - 1
        century = str(century) + '00'
        if century == '000':
            century = '1'
        print(century)

# Final BCE/CE cleanup, now not just on centuries
# This loop gets rid of anything remaining that straddles BCE/CE since this is causing problems
for row, i in enumerate(dai['Date']):
    if len(re.findall("\d+", i)) == 2 and re.search("v\.", i) != None and re.search("n\.", i) != None:
        century_start = '1'
        century_end = re.sub("(\d+)(.*?)(\d+)(.*)", "\\3", i)
        date = century_start + "-" + century_end
        dai['Date'].loc[row] = date

# And single dates in BCE are eliminated
for row, i in enumerate(dai['Date']):
    if len(re.findall("\d+", i)) == 1 and re.search("v\.", i) != None:
        dai['Date'].loc[row] = '0'

# All date ranges that are BCE only
for row, i in enumerate(dai['Date']):
    if len(re.findall("\d+", i)) == 2 and re.search("v\.", i) != None and re.search("n\.", i) == None:
        dai['Date'].loc[row] = '0'

# This one, having solved the '\n's actually replaces everything but the date range
# It works surprisingly well. Previously I ran this at the start of the filtering
# sequence, but it turns out to work much better if it comes at the very end.
for row, i in enumerate(dai['Date']):
    dai['Date'].loc[row] = re.sub("(\d+-\d+|\d+)(.*)", "\\1", i)

# The one category of string artifact that seems to get through the previous round of re filters 
# and messes with the next bit of code in the process, is '.ca' (and 'Ende', as it turns out), 
# so these are removed at this stage
for row, i in enumerate(dai['Date']):
    dai['Date'].loc[row] = re.sub("(.*)ca.\s(.*)|(.*)(Ende\s)(.*)", "\\2", i)

# This loop deals with an artifact created by previous regex steps. Getting the regex order is finnicky so it applies
# to 99% of cases. It produces these strings where 00 and 99 have been added to normal date ranges
# It thinks they are centuries. Anyway, the easiest solution is to ID these and to correct them by 
# subtracting 2 characters from the end of the string. This does that.
for row, i in enumerate(dai['Date']):
    if len(re. findall("\d+", i)) == 2:
        first = re.sub("(\d+)(.*?)(\d+)(.*)", "\\1", i)
        second = re.sub("(\d+)(.*?)(\d+)(.*)", "\\3", i)
        if len(first) > 3:
            first = first[:-2]
            # A previous step seems to -1 these (but not other ranges), so I correct here
            first = int(first) + 1
            first = str(first)
        if len(second) > 3:
            second = second[:-2]
            second = int(second) + 1
            second = str(second)
        newrange = first+'-'+second
        dai['Date'].loc[row] = newrange

# Create two columns from the Date column
dai['Date Min'] = ''
dai['Date Max'] = ''

# This separates dateranges that remain into useful elements for date min and max
# the len business below is simply to see how many dates we are working with
# So len 1 means there is just a single year, len 0 means there is not a meaningful year
# and len 2 is a standard date range
for row, i in enumerate(dai['Date']):
    if len(re. findall("\d+", i)) == 0:
        dai['Date Min'].loc[row] = ''
        dai['Date Max'].loc[row] = ''
    if len(re. findall("\d+", i)) == 1:
        dai['Date Min'].loc[row] = i
        dai['Date Max'].loc[row] = i
    if len(re. findall("\d+", i)) == 2:
        dai['Date Min'].loc[row] = re.sub("(\d+)(.*?)(\d+)(.*)", "\\1", i)
        dai['Date Max'].loc[row] = re.sub("(\d+)(.*?)(\d+)(.*)", "\\3", i)

# We have some detritus that will prevent a successful conversion of the column to int
# This removes that
for row, i in enumerate(dai['Date Min']):
    if re.findall("(vor\s|\(|-)(\d+)", i) != None:
        dai['Date Min'].loc[row] = re.sub("(vor\s|\(|-)(\d+)", "\\2", i)

for row, i in enumerate(dai['Date Max']):
    if re.findall("(vor\s|\(|-)(\d+)", i) != None:
        dai['Date Max'].loc[row] = re.sub("(vor\s|\(|-)(\d+)", "\\2", i)

# The dates in this row somehow survived everything and generated a bad MinMax Date range
# Just drop it here
dai.drop(index=dai[dai['ID'] == 16989].index, inplace=True)
dai.drop(index=dai[dai['Date Min'] == ''].index, inplace=True)
dai.drop(index=dai[dai['Date Min'] == '0'].index, inplace=True)

In [4]:
# Gotta reset index to facilitate looping
dai.reset_index(inplace=True,drop=True)

In [5]:
# Conversion of strings in these columns to integers
dai = dai.astype({"Date Min": int, "Date Max": int})

In [6]:
# Some final weeding out of BCE related entries
# First by getting rid of anything where DateMin is greater than DateMax
for row, i in enumerate(dai['Date Min']):
    if i > dai['Date Max'].loc[row]:
        dai.drop([row], inplace=True)
        
dai.reset_index(inplace=True,drop=True)

In [7]:
# Then removing anything with Republik under ruler
dai = dai[dai['Issuer'] != 'Republik']
dai = dai[dai['Date Min'] > 324]
dai = dai[dai['Date Max'] < 751]


# There is a strange phenomenon where 4 entries of Roman emperors have date ranges transposed,
# appearing to be much later than they are (e.g., 600-699). I simply remove them here
# esp since none would fall into our date range
dai = dai[dai['Issuer'] != 'Antoninus Pius']
dai = dai[dai['Issuer'] != 'Domitianus']
dai = dai[dai['Issuer'] != 'Commodus']
dai = dai[dai['Issuer'] != 'Nerva']
dai.reset_index(inplace=True,drop=True)

## Linking to other datasets and producing group/find dfs

In [8]:
# And here we will link Place to Find information, specifically for geogrpahic coords
places = pd.read_csv('afe_places.csv')
places = places.rename(columns={"ID": "place_id"})
#places.head()

In [9]:
# merge dai (output of last section) with places. produces daip (though I suppose I could keep 
# the name as dai---oh well.)
daip = pd.merge(dai, places,  how='left', left_on=['place_id'], right_on = ['place_id'])

In [10]:
# Need to import the Fund Ort df in order to get type_find
# then we merge with existing daip
fundort = pd.read_csv('afe_fundort.csv')
daip = pd.merge(daip, fundort,  how='left', left_on=['afe_find_id'], right_on = ['ID'])
daip = daip.drop(['ID_y', 'Name', 'Link'], axis=1)

In [11]:
# We begin from detailed finds df, and implicitly at this level of ontology
# we are dealing with single coins. We will be summing these to produce finds
# at that point the qunatities will turn into aggregates
# So this creates those quantities
daip['Quantity'] = 1

In [13]:
# Here, the goal is to create groups by summing the Quantity column along multiple indexes
group = daip.groupby(['Location','afe_find_id','Mint','Denomination','Date Min','Date Max','Issuer'])["Quantity"].apply(lambda x : x.astype(int).sum())
# The process yields a Pandas series. That series should be turned into a df
group = group.to_frame()
# That df needs to level out the indexes back into normal values.
# Each index value simply becomes a repeated value in the relevant cells
group = group.reset_index(level=['Location','afe_find_id','Date Min', 'Date Max', 'Denomination','Mint','Issuer'])

In [14]:
# We need to create distinct group numbers. This covnerts the df index into that.
group = group.reset_index()

In [15]:
# This turns those group numbers into display labels for the map
group['coin_group_id'] = ''
for row, i in enumerate(group.index):
    group['coin_group_id'].loc[row] = 'AFE-' + str(i +1)

/opt/homebrew/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [16]:
# we need to rename a bunch of these columns
group = group.rename(columns={"index": "group_id", "" "Location": "name", "Issuer" : "ruler", "Denomination" : "denomination",\
                     "Quantity" : "num_coins", "Mint" : "mint", "Date Min" : "start_year", "Date Max" : "end_year"})

In [17]:
# Now there are some columns where values need to be added. I am keeping the excavation year column blank for now
today = datetime.date.today()
group['created'] = today.strftime("%m-%d-%Y")
group['imported'] = today.strftime("%m-%d-%Y")
group['owner'] = 'DAI'

In [18]:
# we need to convert denominations to our format before we assign metals
# This was taken almost wholesale from Lee's CHRE code

# For future reference, to find all denoms
# group.denomination.unique()

denomination_conversion = {
    'Denarius':'denarius', 'Semis':'semissis',
       'Aureus':'aureus', 'AE':'uncertain (bronze)', 'Antoninianus':'radiate or nummus (UK find)',
       'Follis':'follis','AE2':'AE2 (5.15g)', 'Drachme':'drachm', 'AV':'AV', 'Maiorina':'follis',
       'AE3':'AE3 (2.58g)', 'Solidus':'solidus', 'Silber':'uncertain (silver)', 'Siliqua':'siliqua', 
    'AE4':'AE4 (1.23g)', 'Tremissis':'tremissis', '10 Num' : '10 nummi', '2 Solidi' : '2 solidi',
    'Siliqua (reduziert)' : 'reduced siliqua', 'Miliarensis' : 'miliarensis'
}

obsolete_denominations = ['Sestertius', 'As', 'Tetradrachme', 'Centenionalis', 'Dupondius',  'Quadrans', 'Doppelsestertius', 'Dupondius / As']

# actual conversion of denominations to FLAME style
group = group[~group['denomination'].isin(obsolete_denominations)]
new_list = group['denomination'].fillna('Uncertain').apply(lambda x:denomination_conversion[x])
group['denomination'] = new_list

In [19]:
# We need to convert denominations to metal
# Generate metal
metal_conversion = {
   'denarius':'silver',
    'semissis':'gold',
    'aureus':'gold',
    'uncertain (bronze)':'bronze',
    'radiate or nummus (UK find)':'bronze',
    'follis':'bronze', 
    'AE2 (5.15g)':'bronze',
    'drachm':'silver',
    'AV':'gold',
    'AE3 (2.58g)':'bronze',
    'solidus':'gold',
    'uncertain (silver)':'silver',
    'siliqua':'silver',
    'AE4 (1.23g)':'bronze',
    'tremissis':'silver',
    '10 nummi':'bronze',
    '2 solidi':'gold',
    'reduced siliqua':'silver',
    'miliarensis':'silver'
}

other_metals = []
group['metal'] = ""
group = group[~group['metal'].isin(other_metals)]
new_list = group['denomination'].fillna('Uncertain').apply(lambda x:metal_conversion[x])
group['metal'] = new_list

In [20]:
# Now to convert mints.
#group.mint.unique()
mint_conversion = {
    'Roma':'Roma', 'Alexandria':'Alexandria ad Aegyptum', 'Uncertain mint':'Unknown', 'Constantinopolis':'Constantinople',
       'Unofficial mint':'Unknown', 'Siscia':'Siscia', 'Thessalonica':'Thessalonika', 'Londinium':'Londinium',
       'Treveri':'Colonia Augusta Treverorum', 'Lugdunum':'Lugdunensium', 'Ticinum':'Ticinum', 'Aquileia':'Aquileia', 'Colonia CAA':'Unknown',
       'Antiochia':'Antioch', 'Emerita':'Emerita', 'Lycia':'Unknown (East Roman)', 'Cyzicus':'Kyzikos', 
    'Roma / Lugdunum':'Roma or Lugdunum', #new mint
       'Sirmium':'Sirmium', 'Eastern mint':'Unknown (East Roman)', 'Gallia':'Unknown (Gaul)', 'Unidentified mint':'Unknown',
       'Laodicea ad Mare':'Laodicea ad Mare', #new mint
    'Mediolanum':'Mediolanum', 'Ravenna':'Ravenna', 'Roma / Tarraco (?)':'Roma or Tarracona', #new mint
       'Africa':'Unknown (Africa)', #new mint
    'Hispania':'Unknown (Iberia)', 'Colonia Caesaraugusta':'Cesaraugusta', 'Greek East':'Unknown (East Roman)',
    'Münzstätte nicht bekannt':'Unknown (Germany)', 'Östliche Münzstätte' : 'Unknown (Germany)',
    '\n\n\n\n\n\n\nDetailed Result\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nID: 11443\n\n\nName\nValue\n\n\nID11443LocationBad Sulza, Bad SulzaStatusOfficialDenominationFollisIssuerConstans':'Bad Sulza',
    '\n\n\n\n\n\n\nDetailed Result\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nID: 11444\n\n\nName\nValue\n\n\nID11444LocationBad Sulza, Bad SulzaStatusOfficialDenominationFollisIssuerConstantius II.':'Bad Sulza',
    'Irreguläre Münzstätte' : 'Unknown (Germany)', 'Sicilia' : 'Sicily', 'Syrien' : 'Unknown (Greater Syria)',
    'Arelate' : 'Arelato', 'Heracleia' : 'Heraclea'
}

obsolete_mints = ['Nicomedia']

group = group[~group['mint'].isin(obsolete_mints)]
new_list = group['mint'].fillna('Uncertain').apply(lambda x:mint_conversion[x])
group['mint'] = new_list

In [21]:
# final regex cleanup of some ruler names, which went through the original regex process (see stages above)
# in the scraping and cleaning stages
# And picked up some odd elements
x = lambda a : re.sub('(.*)(Date)(.*)|(\\n.*)', '\\1', a)
group['ruler'] = group['ruler'].map(x)



#for i in group['ruler']:
#    print(x(i))



In [64]:
group.to_csv(path_or_buf='final_afe_groups.csv')

### Create Finds

In [50]:
# Leaving values NaN in any row of daip means that when you create a new df, they will
# be dropped. You need to provide some value
#daip[['Fundkategorie','References']] = daip[['Fundkategorie','References']].fillna('Unknown')
daip['Fundkategorie'] = daip['Fundkategorie'].fillna('Unknown')

In [59]:
daip.head()

,ID_x,place_id,afe_find_id,Location,Status,Denomination,Issuer,Mint,Date,References,Remarks,Bibliography,Date Min,Date Max,name,lat,long,Fundkategorie,Quantity
0,289,34,2259,"Altkönig, Altkönig",No status,Siliqua,Constantinus III.,Lugdunum,407-408,NaN,Nach Foto bestimmt.Weight1.6,"Böhme, Altkönig, Der ""Altkönig"" im Taunus als ...",407,408,Altkönig,50.212,8.482,Einzelfund,1
1,3522,169,203,"Laatzen, Laatzen Schatzfund",Official,Miliarensis,Constantius II.,Constantinopolis,351-355,"Coh., 326 RIC 8, 100Mint mark-/-//C·AWeight4.5...",\n\n\n\nDetailed Result\n\n\n\n\n\n\n\n\n\n\n\...,"FMRD VII, Die Fundmünzen der Römischen Zeit in...",351,355,Laatzen,52.315,9.797,Schatzfund,1
2,3523,169,203,"Laatzen, Laatzen Schatzfund",Official,Siliqua (reduziert),Constantius II.,Arelate,355-363,"Coh., 343 RIC 8, 261/291Mint mark-/-//SCONWeig...",\n\n\n\nDetailed Result\n\n\n\n\n\n\n\n\n\n\n\...,"FMRD VII, Die Fundmünzen der Römischen Zeit in...",355,363,Laatzen,52.315,9.797,Schatzfund,1
3,3524,169,203,"Laatzen, Laatzen Schatzfund",Imitation,Silber,Constantinus II.,Irreguläre Münzstätte,353-360,"RIC 8, wie (Lug) 216/217 Coh., wie 341Secondar...",Von Berger als barbarische Nachahmung ausgewie...,"FMRD VII, Die Fundmünzen der Römischen Zeit in...",353,360,Laatzen,52.315,9.797,Schatzfund,1
4,3525,169,203,"Laatzen, Laatzen Schatzfund",Official,Miliarensis,Iulianus II.,Lugdunum,360-363,"RIC 8, 209Mint mark-/-//LVG","Datierung nach RIC (abweichend vom FMRD-Band, ...","FMRD VII, Die Fundmünzen der Römischen Zeit in...",360,363,Laatzen,52.315,9.797,Schatzfund,1


In [60]:
# We have a final problem. The original scrape and regex cut the last letter off of some finds
# This is the result of having to filter through huge variation in string patterns
# I don't think an ideal pattern matching formula exists, so rather than play with that
# We just reinput them from afe_fundort
fundort = pd.read_csv('afe_fundort.csv')
# Drop nans
fundort = fundort.dropna(subset=['ID'])
fundort = fundort.reset_index(drop=True)
# Convert to int, not str. Find IDs are ints
fundort['ID'] = fundort['ID'].astype(int)
def find_corrector(find_id):
    find_name = fundort[fundort['ID']==find_id]['Name'].values[0]
    return find_name
daip['Location'] = daip['afe_find_id'].map(find_corrector)

In [61]:
# Here we create finds using the same method.
# We can fill in information in a subsequent step, by joining between the two
#find = daip.groupby(['afe_find_id','Location','Fundkategorie','lat','long','References'])["Quantity"].apply(lambda x : x.astype(int).sum())
find = daip.groupby(['afe_find_id','Location','Fundkategorie','lat','long'])["Quantity"].apply(lambda x : x.astype(int).sum())
# The process yields a Pandas series. That series should be turned into a df
find = find.to_frame()
# That df needs to level out the indexes back into normal values.
# Each index value simply becomes a repeated value in the relevant cells
#find = find.reset_index(level=['afe_find_id','Location','Fundkategorie','lat','long','References'])
find = find.reset_index(level=['afe_find_id','Location','Fundkategorie','lat','long'])

In [64]:
find['year_found'] = '1700'
find['year_found_end'] = today.strftime("%Y")
find['comments'] = ''
find['imported'] = today.strftime("%m-%d-%Y")
find['references'] = ''
for row, i in enumerate(find['afe_find_id']):
    find['references'].loc[row] = 'http://afe.dainst.org/findspot?afeid=' + str(find['afe_find_id'].loc[row])
find['owner'] = 'DAI'
find['created'] = today.strftime("%m-%d-%Y")

/opt/homebrew/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [65]:
# We create all of the columns we can.
# The only one not possible yet is startDate and endDate, which requires
# work to be done on coin groups (return to it below)
find['hoard?'] = find['Fundkategorie'] == 'Schatzfund'
find['single?'] = find['Fundkategorie'] == 'Einzelfund'
find['excavation?'] = find['Fundkategorie'] == 'Kollektivfund'
find['excavation?'] = find['Fundkategorie'] == 'Grabfund'


In [66]:
find = find.rename(columns={"afe_find_id":"find_number","Location":"Name","Fundkategorie":"type_find","Quantity":"num_coins"})

In [67]:
# convert find types from German to English. Some additional work needed for categories that don't fit
# e.g., kollektivfund
find_conversion = {
    'Einzelfund':'Single Find',
    'Schatzfund':'Hoard Find',
    'Excavation':'Excavation Find'
}


#new_list = find['type_find'].fillna('Unbekannt').apply(lambda x:find_conversion[x])
find['type_find'] = new_list

for row, i in enumerate(find['type_find']):
    if find['num_coins'].loc[row] == 1 and i != ('Single Find'or'Hoard Find'):
        find['type_find'].loc[row] = 'Single Find'
    elif find['num_coins'].loc[row] > 1 and i != ('Single Find'or'Hoard Find'):
        find['type_find'].loc[row] = 'Hoard Find'
    else:
        continue

In [68]:
# Finds df is missing certainty values, so we add them here too
# This is not something AFE provides insight on, so we assign our highest value
find['cf_custom_region_vague'] = '2'

In [69]:
# Here the goal is to relink the finds to their component detailed coin finds
# These have been agglomerated and lost in the above summing process
# We just query the daip df by afe_find_id and put it into a new column
our_finds = list(find['find_number'])
biggest = max(find['num_coins'])
all_coins = []
# The loop builds a list of lists for each find (all_coins)
for i in our_finds:
    coins = []
    for y in range(biggest):
        try:
            coin = daip[daip['afe_find_id']==i]['ID_x'].values[y]
            coins.append(coin)
        except:
            continue
    all_coins.append(coins)
# The new column is made from the all_coins list of lists  
find['detailed_find_ids'] = all_coins

In [70]:
# So we make a function to take all those coins indexed to finds and turn them into URLs
# These are mapped onto a new column, group_refs
# That is then concated with references
def det_fin_adder(*list_finds):
    find_citations = ''
    for i in list_finds:
        for y in i:
            find_citations = find_citations + ' || http://afe.dainst.org/detailedresult?l=en&link=' + str(y)
    return find_citations

find['group_refs'] = find['detailed_find_ids'].map(det_fin_adder)
find['references'] = find['references'] + find['group_refs']

# Then we drop the previous columns in favour of just references
find = find.drop(columns=['detailed_find_ids','group_refs'])

In [71]:
find.head(100)

,find_number,Name,type_find,lat,long,num_coins,year_found,year_found_end,comments,imported,references,owner,created,hoard?,single?,excavation?,cf_custom_region_vague
0,201,Schmalstede II (Brandgrab 136),Single Find,54.20000,10.03300,1,1700,2023,,06-15-2023,http://afe.dainst.org/findspot?afeid=201 || ht...,DAI,06-15-2023,False,False,True,2
1,203,Laatzen (Hortfund),Hoard Find,52.31500,9.79700,4,1700,2023,,06-15-2023,http://afe.dainst.org/findspot?afeid=203 || ht...,DAI,06-15-2023,True,False,False,2
2,212,Divitz,Single Find,54.32500,12.68300,1,1700,2023,,06-15-2023,http://afe.dainst.org/findspot?afeid=212 || ht...,DAI,06-15-2023,False,True,False,2
3,214,Demern,Single Find,53.74200,10.98700,1,1700,2023,,06-15-2023,http://afe.dainst.org/findspot?afeid=214 || ht...,DAI,06-15-2023,False,True,False,2
4,216,Kittendorf,Single Find,53.62500,12.90400,1,1700,2023,,06-15-2023,http://afe.dainst.org/findspot?afeid=216 || ht...,DAI,06-15-2023,False,False,True,2
5,218,Radewitz,Single Find,53.29500,14.15500,1,1700,2023,,06-15-2023,http://afe.dainst.org/findspot?afeid=218 || ht...,DAI,06-15-2023,False,True,False,2
6,221,Deersheim I,Single Find,51.98300,10.78100,1,1700,2023,,06-15-2023,http://afe.dainst.org/findspot?afeid=221 || ht...,DAI,06-15-2023,False,False,True,2
7,222,Altenhausen II,Single Find,52.26200,11.25400,1,1700,2023,,06-15-2023,http://afe.dainst.org/findspot?afeid=222 || ht...,DAI,06-15-2023,False,True,False,2
8,435,Quedlinburg II,Hoard Find,51.78800,11.15000,3,1700,2023,,06-15-2023,http://afe.dainst.org/findspot?afeid=435 || ht...,DAI,06-15-2023,False,True,False,2
9,745,Berlin-Friedrichshagen,Single Find,52.45052,13.62463,1,1700,2023,,06-15-2023,http://afe.dainst.org/findspot?afeid=745 || ht...,DAI,06-15-2023,False,True,False,2


In [72]:
find.to_csv(path_or_buf='final_afe_finds.csv')